In [ ]:
# AFTER THIS SCRIPT RUNS: 
# Now copy new_cal.csv to\\nnorsogfas052\Filepolling\Bwinbprd1\AGENCY_BUSINESS_SALES.csv so that BW can load data. If you don't do this BW won't have this new calendar you just created!!!
# during pilot use this file instead: AGENCY_BUSINESS_SALES_inclPilotChanges.csv
# backup the ace form by saving (a copy!!!!!!!!!) to drive - '~/Interim Solution BAU Data Updates/Ace_vkp8_data/previously_loaded/YYYYMMDD_HHMM ...'

In [ ]:
import pandas as pd
import numpy as np
import datetime
import re

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

from google.colab import drive
drive.mount('/content/drive')


Authenticated
Mounted at /content/drive


In [ ]:
!cp /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/ARace/R/ARace_Functions.R /content/ARace_Functions.R
!cp /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/AGENCY_BUSINESS_SALES.feather /content/AGENCY_BUSINESS_SALES.feather

In [ ]:
%%bigquery --project gcp-wow-ent-im-tbl-prod valid_articles

#standardSQL
select distinct lpad(Article, 18, '0') as Article
from `gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_article_v`

In [ ]:
%%bigquery --project gcp-wow-ent-im-tbl-prod valid_vendors

#standardSQL
select distinct lpad(Vendor, 10, '0') as Vendor
from `gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_vendor_v`

In [ ]:
# vendor '0000000000' is a valid vendor in BW so force it
vend_0 = pd.DataFrame({'Vendor':['0'.zfill(10)]})
valid_vendors = valid_vendors.append(vend_0, ignore_index = True)

In [ ]:
%%bigquery --project gcp-wow-ent-im-tbl-prod valid_salesorgs

#standardSQL
select distinct SalesOrganisation
from `gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_site_v`

In [ ]:
#Get the 'Weekly Agency Update Data.gsheet'
#=========================================================================================================================================================================================================================
!pip install gspread --upgrade # Uninstall gspread-3.0.1 with 3.7.0
import gspread
from oauth2client.client import GoogleCredentials as GC
GoogleSheets = gspread.authorize(GC.get_application_default())
import gspread_dataframe 

## for some reason this doesn't work:
#wb = GoogleSheets.open('/content/drive/My Drive/Interim Solution BAU Data Updates/Ace_VPK8_data/Weekly Agency Update Data')#.sheet1
# so do this:
wb = GoogleSheets.open_by_url('https://docs.google.com/spreadsheets/d/1L5mrV_bDvdWjOlPt07ggMgRbC0sq23EkBhorc5aAhi8/edit#gid=0')


  Attempting uninstall: gspread
    Found existing installation: gspread 3.0.1
    Uninstalling gspread-3.0.1:
      Successfully uninstalled gspread-3.0.1


In [ ]:
def function_get_vkp8_data(wsName='_VKP8'):
  mWB = wb.worksheet(wsName)
  dat = mWB.get_all_values()
  df = pd.DataFrame(dat, columns=['Article', 'SalesOrg', 'PL', 'Vendor', 'Valid_from'	, 'Valid_to'	, 'New_GP_perc'])
  df = df.iloc[1:] # removes column header row from 1st data row

  df = df[ (df.PL.isna()) | (df.PL == '')] # for vkp8 only use national prices - so PL blank

  # set column types
  def re_dtype(df, col_dtypes):
    for k,v in col_dtypes.items():
      if k == 'str':
        for vv in v:
          df[vv] = df[vv].astype(str)
      if k == 'int':
        for vv in v:
          df[vv] = df[vv].astype(str).astype(int)
      if k == 'date':
        for vv in v:
          df[vv] = df[vv].astype(str)
          df = df[ (~df[vv].isna()) & (df[vv] != '')]
          #df[vv] = pd.to_datetime(df[vv], format="%d/%m/%Y") #, %H:%M:%S")
          #df[vv] = datetime.datetime.strptime(df[vv], "%d/%m/%Y").date()
          df[vv] = [datetime.datetime.strptime(x, "%d/%m/%Y").date() for x in df[vv]]
      if k == 'float':
        for vv in v:
          df[vv] = df[vv].astype(str).astype(float)
    return df

  mcol_dtypes = {'str': ['Article', 'SalesOrg', 'PL', 'Vendor'],
                'date': ['Valid_from'	, 'Valid_to'],
                'float': ['New_GP_perc']}
  df = re_dtype(df,mcol_dtypes)
  df = df[['Article', 'SalesOrg', 'Vendor',  'Valid_from', 'Valid_to', 'New_GP_perc']]
  df = df.rename(columns={"New_GP_perc":"Commission_Rate", "Valid_from":"start_date", "Valid_to":"end_date"})
  df['Article'] = [x.zfill(18) for x in df['Article']]

  #df['Vendor'] = [re.sub('\n', '',x) for x in df['Vendor']]
  #df['Vendor'] = [x.zfill(10) for x in df['Vendor']]
  df['Vendor'] = ['0'.zfill(10) for x in df['Vendor']]

  df['Commission_Rate'] = df['Commission_Rate']/100
  df['Commission_Rate'] = round(df['Commission_Rate'],3)

  df = df.dropna()

  return df

In [ ]:
supers_vkp8_data = function_get_vkp8_data()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
def function_get_ace_data(wsName='_ACE Data SM', mso='1005'):
  mWB = wb.worksheet(wsName)
  dat = mWB.get_all_values()
  df = pd.DataFrame(dat, columns=['ActionRequired', 'X0', 'Article', 'X1', 'Article_Description', 'X2', 'X3', 'X4', 'X5', 'X6', 'GTIN', 'X7', 'X8', 'X9', 'Vendor_Number', 'X10', 'X11', 'Vendor_Name', 'GST_Rate', 'Net_Cost', 'Sell_Price_Incl_GST', 'Sell_Price_Excl_GST', 'Calculated_Commission_Rate', 'Master_Commission_Rate', 'Valid_From_Date', 'Valid_To_Date'])

  df = df.iloc[1:] # removes column header row from 1st data row

  df = df[ (~df.Article.isna()) & (df.Article != '')] 

  df['SalesOrg'] = mso
  df = df[['Article', 'SalesOrg', 'Vendor_Number',  'Valid_From_Date', 'Valid_To_Date', 'Master_Commission_Rate']]
  df = df.rename(columns={"Vendor_Number":"Vendor","Master_Commission_Rate":"Commission_Rate", "Valid_From_Date":"start_date", "Valid_To_Date":"end_date"})

  # trim leading and trailing whitespace before padding zero's
  df['Article'] = [x.strip() for x in df.Article]
  df['Article'] = [x.zfill(18) for x in df['Article']]

  # trim leading and trailing whitespace before padding zero's
  df['Vendor'] = [x.strip() for x in df.Vendor]
  df['Vendor'] = ['0'.zfill(10) for x in df['Vendor']]

  for mcol in ['start_date', 'end_date']:
    df[mcol] = [datetime.datetime.strptime(x, "%d/%m/%Y").date() for x in df[mcol]]

  df['Commission_Rate'] = [re.sub('%', '',x) for x in df['Commission_Rate']]
  df['Commission_Rate'] = df['Commission_Rate'].astype(str).astype(float)
  df['Commission_Rate'] = df['Commission_Rate']/100
  df['Commission_Rate'] = round(df['Commission_Rate'],3)

  df = df.dropna()

  return df


In [ ]:
# Removals get a commission rate of 666%: 6.66.
supers_ace_data = function_get_ace_data(wsName='_ACE Data SM', mso='1005')
nz_ace_data = function_get_ace_data(wsName='_ACE Data NZ', mso='2010')
bigw_ace_data = function_get_ace_data(wsName='_ACE Data BigW', mso='1060')


In [ ]:
def checkIn(x,y):
  each_check = [x in y for x in x]
  return all(each_check)


def checkDataAccuracy(df):
  checkVendors = checkIn(list(df['Vendor']), list(valid_vendors['Vendor']))
  checkArticles = checkIn(list(df['Article']), list(valid_articles['Article']))
  checkSalesOrg = checkIn(list(df['SalesOrg']), list(valid_salesorgs['SalesOrganisation']))

  print('Valid Vendor check:', checkVendors)
  print('Valid Articles check:', checkArticles)
  print('Valid SalesOrg check:', checkSalesOrg)

  return(all([checkVendors,checkArticles,checkSalesOrg]))

#myDataCheck = checkDataAccuracy(all_change_request_data)
myDataCheck = checkDataAccuracy(supers_ace_data.append(nz_ace_data).append(bigw_ace_data))

# if data issue then raise error (stopping further code execution)
if not myDataCheck:
  raise ValueError('data values invalid.')



Valid Vendor check: True
Valid Articles check: True
Valid SalesOrg check: True


In [ ]:
all_change_request_data = supers_vkp8_data \
                          .append(supers_ace_data) \
                          .append(nz_ace_data) \
                          .append(bigw_ace_data)

In [ ]:
# save to feather for R to import. This instead of %R -i all_change_request_data avoids date type data issues
all_change_request_data.reset_index(drop=True).to_feather('all_change_request_data.feather')
all_change_request_data.reset_index(drop=True).to_csv('all_change_request_data.csv')

In [ ]:
# activate R magic
#!pip install rpy2==3.2.2
!pip install rpy2
%load_ext rpy2.ipython

In [ ]:
%%R #-i all_change_request_data

library(tidyverse)
#if (!require("feather")) install.packages("feather")
install.packages("feather")
library(feather)

source("ARace_Functions.R")
#all_change_request_data <- feather::read_feather('all_change_request_data.feather')
all_change_request_data <- read_csv("all_change_request_data.csv") %>%
  #dplyr::select(-X1) %>%
  .[,-1] %>%
  dplyr::mutate(SalesOrg=as.character(SalesOrg))


print("importing starting cal")
starting_calendar <- feather::read_feather("AGENCY_BUSINESS_SALES.feather") %>%
  dplyr::mutate(start_date=as.Date(start_date, format="%Y%m%d")
                , end_date=as.Date(end_date, format="%Y%m%d"))
# starting_calendar <- read_csv("AGENCY_BUSINESS_SALES.csv") %>%
#   dplyr::select(-X1) %>%
#   dplyr::mutate(SalesOrg=as.character(SalesOrg))
  
print(head(starting_calendar))
print(head(all_change_request_data))

print(str(starting_calendar))
print(str(all_change_request_data))

print("Making new calendar")
new_cal <- get_new_calendar(starting_calendar, all_change_request_data, flag_check_Commission_Rates=FALSE, flag_push_supers_data=TRUE) %>%
  dplyr::mutate(Commission_Rate=Commission_Rate/100)
# new_cal <- get_new_calendar(starting_calendar, all_change_request_data, flag_check_Commission_Rates=FALSE, flag_push_supers_data=TRUE) %>%
#   dplyr::mutate(Commission_Rate=Commission_Rate)

# If commission rate = 666%: 6.66 then delete record.
new_cal <- new_cal %>%
dplyr::filter(Commission_Rate != 6.66)

saveRDS(new_cal, "new_cal.rds")
feather::write_feather(new_cal, 'new_cal.feather')
write.csv(new_cal, "new_cal.csv", row.names = FALSE, quote = FALSE)

saveRDS(starting_calendar, "starting_calendar.rds")
feather::write_feather(starting_calendar, 'starting_calendar.feather')
write.csv(starting_calendar, "starting_calendar.csv", row.names = FALSE, quote = FALSE)

print("Checking new calendar has >= n rows of starting calendar")
checkRows <- nrow(new_cal)-nrow(starting_calendar)
if( checkRows < 0 ){stop("new calc has less records than old cal. Is this correct?")}

print("Checking that no changes in entries relate to article not in change request")
# check that no changes in entries relate to article not in change request
new_entries <- new_cal  %>%
  dplyr::mutate(Commission_Rate=round(Commission_Rate, 3)) %>%
  dplyr::mutate(start_date=as.Date(start_date, format="%Y%m%d")
  , end_date=as.Date(end_date, format="%Y%m%d")) %>%
  dplyr::full_join(starting_calendar %>%
                     dplyr::mutate(Commission_Rate=round(Commission_Rate, 3)) %>%
                     dplyr::mutate(idx=1:nrow(starting_calendar))
                   , by=c("Article", "SalesOrg", "Vendor", "start_date", "end_date", "Commission_Rate")) %>%
  dplyr::filter(is.na(idx))
which_wrong <- which( !(new_entries$Article %in% all_change_request_data$Article) )
if( length(which_wrong)>0 ){print(new_entries[which_wrong,]); stop("An articles has changed that isn't in change requests. Execute new_entries[which_wrong,] to see articles of concern.")}

saveRDS(new_entries[which_wrong,], "newEntries_whichWrong.rds")
write.csv(new_entries[which_wrong,], "newEntries_whichWrong.csv", row.names = FALSE, quote = FALSE)

saveRDS(new_entries, "newEntries.rds")
write.csv(new_entries, "newEntries.csv", row.names = FALSE, quote = FALSE)

print("Run successfully. WOOHOO!!")

R[write to console]: ── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

R[write to console]: ✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

R[write to console]: ── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/feather_0.3.5.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 73812 bytes (72 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]

Rows: 518 Columns: 7
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (2): Article, Vendor
dbl  (3): ...1, SalesOrg, Commission_Rate
date (2): start_date, end_date

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
[1] "importing starting cal"
# A tibble: 6 × 6
  Article            SalesOrg Vendor     start_date end_date   Commission_Rate
  <chr>              <chr>    <chr>      <date>     <date>               <dbl>
1 000000000000000006 1060     0000000000 2019-07-01 9999-12-31           0.15 
2 000000000000000294 1005     0000000000 2019-01-14 2019-12-22           0.225
3 000000000000000294 1005     0000000000 2019-12-23 2020-11-01           0.249
4 000000000000000294 1005     0000000000 2020-11-02 2021-03-11           0.249
5 000000000000000294 1005     0000000000 2021-03-12 2021-06-03           0.25 
6 000000000000000294 1005    

R[write to console]: 0 instances of end_date < start_date found.



NULL


R[write to console]: 0 instances of missing values for article, commission rate or date.



NULL
`summarise()` has grouped output by 'Article', 'SalesOrg'. You can override using the `.groups` argument.
`summarise()` has grouped output by 'Article', 'SalesOrg'. You can override using the `.groups` argument.
[1] "Checking new calendar has >= n rows of starting calendar"
[1] "Checking that no changes in entries relate to article not in change request"
[1] "Run successfully. WOOHOO!!"


### pilot articles (set to_date as 15/9/2021)
Put this here to ensure no above code leads to incorrect 'end_date' for pilot vendors articles.

emails from Jacinta Rodrick:
"
Saturday 11/9 will be a technical go live, where we will put in the code and not switch it on,
Thursday 16/9 will be the business go live. We will be end dating those pilot agency articles with the 15/9 and the new solution will be effective from the 16/9.
"

list of vendors in this gsheet:
https://docs.google.com/spreadsheets/d/1gudE1F4-j7GN1ehj2EIr1q8GNnvSxkyiGPF7ZI01y0E/edit#gid=958548641

In [ ]:
%%bigquery --project gcp-wow-ent-im-tbl-prod pilot_articles

#standardSQL
SELECT distinct lpad(Article, 18, '0') as pilot_articles, '20210915' as pilot_end_date
from `gcp-wow-ent-im-tbl-prod.adp_dm_masterdata_view.dim_article_vendor_purchaseorg_site_v`
where lpad(Vendor, 10, '0') in (
'0067546001',
'0067546057',
'0000669010',
'0074994001',
'0000021530',
'0000021511',
'0001029001',
'0001647001',
'0000669010',
'0083152001',
'0002802001',
'0083106001',
'0074994001'
)


In [ ]:
# purpose here is to take new_cal (which used to be our AGENCY_BUSINESS_SALES.csv polling file)
# change end dates for pilot articles
# then that csv is our new AGENCY_BUSINESS_SALES.csv
# but still make sure we save all the old and NEW files to drive in a way where we can still post all changes as if pilot wan't on week to week in a separate file and then only correct these in this step.
# so that if pilot is called off we still have an up to date file to refer to. Hence why we AREN't changing any of the old process.
new_cal_prePilotChanges = pd.read_csv("new_cal.csv", dtype={'Article':'string','SalesOrg':'string','Vendor':'string','start_date':'string','end_date':'string'})
print(new_cal_prePilotChanges.head())
# # commission rate was coming through as 0.0015 instead of 0.15 so just *100 and round 3.
# new_cal_prePilotChanges['Commission_Rate']=round(new_cal_prePilotChanges['Commission_Rate']*100,3)
new_cal_prePilotChanges['Commission_Rate']=round(new_cal_prePilotChanges['Commission_Rate'],3)
print(new_cal_prePilotChanges.head())
new_cal_prePilotChanges.reset_index(drop=True).to_csv('new_cal_prePilotChanges.csv', index=False)

              Article SalesOrg  ...  end_date Commission_Rate
0  000000000000000006     1060  ...  99991231           0.150
1  000000000000000294     1005  ...  20191222           0.225
2  000000000000000294     1005  ...  20201101           0.249
3  000000000000000294     1005  ...  20210311           0.249
4  000000000000000294     1005  ...  20210603           0.250

[5 rows x 6 columns]
              Article SalesOrg  ...  end_date Commission_Rate
0  000000000000000006     1060  ...  99991231           0.150
1  000000000000000294     1005  ...  20191222           0.225
2  000000000000000294     1005  ...  20201101           0.249
3  000000000000000294     1005  ...  20210311           0.249
4  000000000000000294     1005  ...  20210603           0.250

[5 rows x 6 columns]


In [ ]:
new_cal_inclPilotChanges = new_cal_prePilotChanges.merge(pilot_articles, how='left',left_on='Article', right_on='pilot_articles')
# now change end_date if pilot requires that
# if pilot article and end_date > pilot_end_date then pilot_end_date else end_date
new_cal_inclPilotChanges['end_date'] = np.where( (new_cal_inclPilotChanges.pilot_end_date.isna()) | (new_cal_inclPilotChanges.end_date < new_cal_inclPilotChanges.pilot_end_date), 
                                                new_cal_inclPilotChanges.end_date, 
                                                new_cal_inclPilotChanges.pilot_end_date)
# now drop unecessary columns
new_cal_inclPilotChanges.drop(['pilot_end_date','pilot_articles'], axis=1, inplace=True)
new_cal_inclPilotChanges.to_csv('new_cal_inclPilotChanges.csv', index=False)

In [ ]:
%%bash
d=$(TZ=Australia/Sydney date +%Y%m%d_%H%M%S)
mkdir /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d
cp /content/new_cal.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/AGENCY_BUSINESS_SALES.csv
cp /content/new_cal.feather /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/AGENCY_BUSINESS_SALES.feather

cp /content/starting_calendar.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/starting_calendar.csv
cp /content/starting_calendar.feather /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/starting_calendar.feather

cp /content/all_change_request_data.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/change_requests.csv
cp /content/all_change_request_data.feather /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/change_requests.feather

cp /content/new_cal.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/AGENCY_BUSINESS_SALES.csv
cp /content/new_cal.feather /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/AGENCY_BUSINESS_SALES.feather

pip freeze > /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/requirements.txt
cp /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/run_agency.ipynb /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/run_agency.ipynb

# now for the pilot files...
cp /content/new_cal_inclPilotChanges.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/Commission_Calendars/$d/AGENCY_BUSINESS_SALES_inclPilotChanges.csv
cp /content/new_cal_inclPilotChanges.csv /content/drive/My\ Drive/Interim\ Solution\ BAU\ Data\ Updates/Ace_VPK8_data/AGENCY_BUSINESS_SALES_inclPilotChanges.csv

echo "Folder '$d' created and files saved."

Folder '20210909_110705' created and files saved.
